In [25]:
from scipy.stats import t
import matplotlib.pyplot as plt
from PIL import Image
import random;
import numpy;
import math;

# Load Path Data

In [26]:
pathMeanTimes = [[],[],[],[],[]];
pathStdOfTimes = [[],[],[],[],[]];
pathDistances = [];
percentOfTimeInEachRegion = [];
P_I = [[],[],[],[],[]];

numberOfPeople = 2;

for personi in range(0,numberOfPeople):

    randForPI=[]
    for randpi in range(0,5):
        randForPI.append(random.uniform(0,1));
        
    #from 1340 Como Avenue, Saint Paul, MN 55108 to 7501-7549 Noble Avenue North, Brooklyn Park, MN 55443
    #via MN-36 and I-694 W. #T = 23min. D = 16.6 mi
    #Get on MN-36 W in Roseville from Snelling Ave
    #7 min (2.9 mi)
    #Take I-35W N and I-694 W to County Rd 152/Brooklyn Blvd in Brooklyn Center. Take exit 33 from I-694 W
    #13 min (12.5 mi)
    #Continue on County Rd 152/Brooklyn Blvd. Drive to Noble Ave N in Brooklyn Park
    #3 min (1.2 mi) 
    pathMeanTimes[0].append(random.normalvariate(1380,80));
    pathStdOfTimes[0].append(random.normalvariate(60,13));
    pathDistances.append(16.6);
    P_I[0].append(randForPI[0]/sum(randForPI));#actual
    percentOfTimeInEachRegion.append([0,0,0,.22,0,.1,0,.1,.26,0,.22,.1]);

    #via I-94. T=26min. D=19.5 mi
    #Get on I-94 W in Union Park from Como Ave and Snelling Ave
    #7 min (2.5 mi)
    #Follow I-94 W to County Rd 152/Brooklyn Blvd in Brooklyn Center. Take exit 33 from I-94 W
    #16 min (15.8 mi)
    #Continue on County Rd 152/Brooklyn Blvd. Drive to Noble Ave N in Brooklyn Park
    #3 min (1.2 mi) 
    pathMeanTimes[1].append(random.normalvariate(1560,95));
    pathStdOfTimes[1].append(random.normalvariate(95,15));
    pathDistances.append(19.5);
    P_I[1].append(randForPI[1]/sum(randForPI));
    percentOfTimeInEachRegion.append([0,0,0,.2,0,0,.03,.14,0,.2,0,.43]);

    #via I-35. T = 28min. D = 22 mi
    #Get on MN-36 W in Roseville from Snelling Ave
    #7 min (2.9 mi)
    #Take I-35W N, US-10 W and MN-610 to Noble Pkwy in Brooklyn Park. Take the Noble Pkwy exit from MN-610 W
    #16 min (16.7 mi)
    #Drive to Noble Ave N
    #5 min (2.4 mi) 
    pathMeanTimes[2].append(random.normalvariate(1680,100));
    pathStdOfTimes[2].append(random.normalvariate(100,16));
    pathDistances.append(22);
    P_I[2].append(randForPI[2]/sum(randForPI));
    percentOfTimeInEachRegion.append([.12,.12,.16,.06,.04,.12,0,0,.17,0,.13,.07]);

    # T=24min. D = 17.1 mi
    #Get on MN-36 W in Roseville from Snelling Ave
    #6 min (2.9 mi)
    #Take I-35W N and I-694 W to Shingle Creek Pkwy in Brooklyn Center. Take exit 34 from I-694 W
    #12 min (11.3 mi)
    #Continue on Shingle Creek Pkwy. Take 69th Ave N to Noble Ave N in Brooklyn Park
    #6 min (2.9 mi) 
    pathMeanTimes[3].append(random.normalvariate(1440,85));
    pathStdOfTimes[3].append(random.normalvariate(85,13));
    pathDistances.append(17.1);
    P_I[3].append(randForPI[3]/sum(randForPI));
    percentOfTimeInEachRegion.append([0,0,0,.27,0,.09,0,.09,.25,0,.21,.09]);

    #via T = 26 min D =19.4 mi
    #Get on MN-280 S from Como Ave and Energy Park Dr
    #7 min (3.1 mi)
    #Take I-94 W to County Rd 152/Brooklyn Blvd in Brooklyn Center. Take exit 33 from I-94 W
    #16 min (15.1 mi)
    #Continue on County Rd 152/Brooklyn Blvd. Drive to Noble Ave N in Brooklyn Park
    #3 min (1.2 mi) 
    pathMeanTimes[4].append(random.normalvariate(1560,90));
    pathStdOfTimes[4].append(random.normalvariate(90,12));
    pathDistances.append(19.4);
    P_I[4].append(randForPI[4]/sum(randForPI));
    percentOfTimeInEachRegion.append([0,0,0,.2,0,0,.03,.14,0,.2,0,.43]); 
    
    

    #print("Path Mean Times in seconds: ");
    #print(pathMeanTimes);
    #print("\nPath Standard Deviation of Times in seconds: ");
    #print(pathStdOfTimes);
    #print("\nPath Distances in miles:");
    #print(pathDistances);
    #print("\nPath Percent in each Region: ");
    #print(percentOfTimeInEachRegion);
    #print("\nP_I(x):");
    #print(P_I);

# Simulation

# Initialize Variables

In [27]:
#initial values
N = 7;
updateInterval = 30;

pi_m = [];
pi_s = [];
numberOfTimesRouteTaken = [];
routeTravelTimes=[]; #stores only history since last update
allRouteTravelTimes=[]; #stores all the history
m=[];
c=[];
tbar=[];
sumtijsquared=[];
v=[];
a=[];
b=[];
tloc=[];
tscale = [];
tdf = [];
ptx=[];
px=[];
pbxt=[];

#two dimensional arrays are not consistent with the load data section. TODO: reformat 
for personi in range(0,numberOfPeople):
    pi_m.append([]);
    pi_s.append([]);
    numberOfTimesRouteTaken.append([0,0,0,0,0]);
    routeTravelTimes.append([[],[],[],[],[]]);
    allRouteTravelTimes.append([[],[],[],[],[]]);
    #different guess for m for each person
    
    case = 1;
    if case==1:
        m.append([random.normalvariate(1350,80),random.normalvariate(1575,90),random.normalvariate(1680,95),random.normalvariate(1440,95),random.normalvariate(1560,65)]);
    elif case==2:
        m.append([random.normalvariate(1150,60),random.normalvariate(1330,45),random.normalvariate(1390,70),random.normalvariate(1220,75),random.normalvariate(1780,65)]);
    else: 
        m.append([random.normalvariate(550,60),random.normalvariate(1230,45),random.normalvariate(1975,70),random.normalvariate(1720,75),random.normalvariate(970,65)]);
    
    c.append([0,0,0,0,0]);
    tbar.append([0,0,0,0,0]);
    sumtijsquared.append([0,0,0,0,0]); 
    v.append([1,1,1,1,1]);
    a.append([0.5,0.5,0.5,0.5,0.5]);
    b.append([0.1,0.1,0.1,0.1,0.1]);   

    #tdistribution params
    tloc.append([950,930,1000,1090,1100]); #mean #This gets set to m so initial values don't matter
    tscale.append([50, 60, 65, 80, 55]); #"stdedv" #initial values shouldn't matter
    tdf.append([1,1,1,1,1]); #degrees of freedom

    ptx.append([0,0,0,0,0]); # probability of time t given route x
    px.append([0,0,0,0,0]); # probability of route x
    pbxt.append([0,0,0,0,0]); # probability of route x given time t


SyntaxError: invalid syntax (<ipython-input-27-599953a90172>, line 36)

# Training

In [22]:
for personi in range(0,numberOfPeople):
    for x in range(0,N):

        #Pick a random path based on P_I
        randomNumber = random.uniform(0,1);
        pathLimit=0;
        for r in range(0,len(P_I)):
            pathLimit = pathLimit+P_I[r][personi];
            if(randomNumber<pathLimit):
                randomPath = r;
                break;
        print("\nPath: "+str(randomPath));

        #Find a random time that it took for that path based on normal distribution for that path
        pathUserTime = random.normalvariate(pathMeanTimes[randomPath][personi],pathStdOfTimes[randomPath][personi]);
        print("User Time in Seconds: "+str(pathUserTime));
        routeTravelTimes[personi][randomPath].append(pathUserTime);
        allRouteTravelTimes[personi][randomPath].append(pathUserTime);
        numberOfTimesRouteTaken[personi][randomPath] = numberOfTimesRouteTaken[personi][randomPath] + 1;
        c[personi][randomPath] = c[personi][randomPath]+1;
        tbar[personi][randomPath] = (tbar[personi][randomPath]*(len(routeTravelTimes[personi][randomPath])-1)+pathUserTime)/len(routeTravelTimes[personi][randomPath]);
        sumtijsquared[personi][randomPath] = sumtijsquared[personi][randomPath] + pow(pathUserTime,2);

        #Calculate actual time in each weather region
        actualTimeInEachRegion = [];    
        for r in range(0,len(percentOfTimeInEachRegion[randomPath])):#r is the region number
            actualTimeInEachRegion.append(percentOfTimeInEachRegion[randomPath][r]*pathUserTime);

        print("Time in each region: "+str(actualTimeInEachRegion));

        print("tbar: " + str(tbar[personi][randomPath]));
        print("c: " + str(c[personi][randomPath]) );
        print("sumtijsquared: " + str(sumtijsquared[personi][randomPath]) );

        #update if we've reached updateInterval
        if(c[personi][randomPath]%updateInterval==0):
            #Update prior hyperparameters
            oldm = m[personi][randomPath];
            oldv = v[personi][randomPath];
            olda = a[personi][randomPath];
            oldb = b[personi][randomPath];

            v[personi][randomPath] = oldv/(1+c[personi][randomPath]*oldv);
            m[personi][randomPath] = (oldm/oldv+c[personi][randomPath]*tbar[personi][randomPath])*v[personi][randomPath]        
            a[personi][randomPath] = olda+c[personi][randomPath]/2;

            b[personi][randomPath] = oldb + (1/2)*(pow(oldm,2)/oldv+sumtijsquared[personi][randomPath]-pow(m[personi][randomPath],2)/v[personi][randomPath]);

            c[personi][randomPath] = 0; 
            tbar[personi][randomPath] = 0;
            sumtijsquared[personi][randomPath] = 0;
            routeTravelTimes[personi][randomPath] = [];

            print("UPDATED HYPER PARAMS \nm: "+str(m[randomPath])+
            "\nv: " + str(v[personi][randomPath]) + 
            "\na: " + str(a[personi][randomPath]) +
            "\nb: " + str(b[personi][randomPath]));

            #Update Tdistribution params

            tloc[randomPath] = m[randomPath];
            tscale[randomPath] = math.sqrt(b[randomPath]*(1+v[randomPath])/(a[randomPath]));
            tdf[randomPath] = 2*a[randomPath];

        #update if we've reached the end
        if x == N-1:
            for i in range(0,len(numberOfTimesRouteTaken[personi])): 
                #Update prior hyperparameters
                oldm = m[personi][i];
                oldv = v[personi][i];
                olda = a[personi][i];
                oldb = b[personi][i];

                v[personi][i] = oldv/(1+c[personi][i]*oldv);
                m[personi][i] = (oldm/oldv+c[personi][i]*tbar[personi][i])*v[personi][i]        
                a[personi][i] = olda+c[personi][i]/2;

                b[personi][i] = oldb + (1/2)*(pow(oldm,2)/oldv+sumtijsquared[personi][i]-pow(m[personi][i],2)/v[personi][i]);

                c[personi][i] = 0; 
                tbar[personi][i] = 0;
                sumtijsquared[personi][i] = 0;
                routeTravelTimes[personi][i] = [];                
                print("Route "+str(i)+" UPDATED HYPER PARAMS \nm: "+str(m[personi][i])+
                "\nv: " + str(v[personi][i]) + 
                "\na: " + str(a[personi][i]) +
                "\nb: " + str(b[personi][i]));

                #Update Tdistribution params

                tloc[personi][i] = m[personi][i];
                tscale[personi][i] = math.sqrt(b[personi][i]*(1+v[personi][i])/(a[personi][i]));
                tdf[personi][i] = 2*a[personi][i];
        

        



Path: 3
User Time in Seconds: 1218.0036316869468
Time in each region: [0.0, 0.0, 0.0, 328.8609805554757, 0.0, 109.62032685182521, 0.0, 109.62032685182521, 304.5009079217367, 0.0, 255.78076265425884, 109.62032685182521]
tbar: 1218.0036316869468
c: 1
sumtijsquared: 1483532.8468025916

Path: 2
User Time in Seconds: 1548.1778521151955
Time in each region: [185.78134225382345, 185.78134225382345, 247.7084563384313, 92.89067112691173, 61.92711408460782, 185.78134225382345, 0.0, 0.0, 263.19023485958326, 0.0, 201.26312077497542, 108.3724496480637]
tbar: 1548.1778521151955
c: 1
sumtijsquared: 2396854.66178002

Path: 3
User Time in Seconds: 1359.638564679744
Time in each region: [0.0, 0.0, 0.0, 367.10241246353087, 0.0, 122.36747082117695, 0.0, 122.36747082117695, 339.909641169936, 0.0, 285.5240985827462, 122.36747082117695]
tbar: 1288.8210981833454
c: 2
sumtijsquared: 3332149.8733669855

Path: 4
User Time in Seconds: 1473.9622468122807
Time in each region: [0.0, 0.0, 0.0, 294.7924493624561, 0.0

# Testing

In [23]:
#LOAD VARIABLES RUN THIS ONCE BEFORE RUNNING TESTING BELOW. THEN YOU CAN RUN TESTING OVER AND OVER AS MANY TIMES
#p(t|x)
numberOfTest = 7;
ci=[];
actualCITI=[];
estimatedCITI=[];
for personi in range(0,numberOfPeople):
    case=1;
    if case == 1:
        ci.append([8,10,13,9,16,12,15,11,10,6,13,11]);
    elif case ==2:
        ci.append([5,20,13,7,22,12,15,6,13,23,11,6]);
    else:
        ci.append([3,25,13,4,26,14,21,27,14,4,19,23]);
    
    actualCITI.append([[],[],[],[],[],[],[],[],[],[],[],[]]); #actual c_i*t_i for all tests
    estimatedCITI.append([[],[],[],[],[],[],[],[],[],[],[],[]]); #estimated c_i*t_i for all tests

In [34]:
for personi in range(0,numberOfPeople):
    for k in range(0,numberOfTest):

        #Pick a random path based on P_I
        randomNumber = random.uniform(0,1);
        pathLimit=0;
        for r in range(0,len(P_I)):
            pathLimit = pathLimit+P_I[r][personi];
            if(randomNumber<pathLimit):
                randomPath = r;
                break;
        print("\nPath: "+str(randomPath));

        #Find a random time that it took for that path based on normal distribution for that path
        pathUserTime = random.normalvariate(pathMeanTimes[randomPath][personi],pathStdOfTimes[randomPath][personi]);
        print("User Time in Seconds: "+str(pathUserTime));
        routeTravelTimes[personi][randomPath].append(pathUserTime);
        allRouteTravelTimes[personi][randomPath].append(pathUserTime);
        numberOfTimesRouteTaken[personi][randomPath] = numberOfTimesRouteTaken[personi][randomPath] + 1;
        c[personi][randomPath] = c[personi][randomPath]+1;
        tbar[personi][randomPath] = (tbar[personi][randomPath]*(len(routeTravelTimes[personi][randomPath])-1)+pathUserTime)/len(routeTravelTimes[personi][randomPath]);
        sumtijsquared[personi][randomPath] = sumtijsquared[personi][randomPath] + pow(pathUserTime,2);


        #Calculate actual time in each weather region
        actualTimeInEachRegion = [];    
        for r in range(0,len(percentOfTimeInEachRegion[randomPath])):
            actualTimeInEachRegion.append(percentOfTimeInEachRegion[randomPath][r]*pathUserTime);

        print("Time in each region: "+str(actualTimeInEachRegion));

        #multiply c_i*t_i
        for regionNum in range(0,len(actualTimeInEachRegion)):
            actualCITI[personi][regionNum].append(ci[personi][regionNum]*actualTimeInEachRegion[regionNum]);
            print("Actual CITI region "+ str(regionNum)+": "+ str(ci[personi][regionNum]*actualTimeInEachRegion[regionNum]))

        print("\nProbabilities for t ="+str(pathUserTime));
        for x in range(0,len(numberOfTimesRouteTaken[personi])):
            print("ROUTE "+str(x)+" df = "+str(tdf[personi][x]));
            #arr = numpy.array(allRouteTravelTimes[x]);
            #std = numpy.std(arr, axis=0);
            #tinterval = (1/20)*std;
            ptx[personi][x] = t.pdf(pathUserTime, tdf[personi][x], tloc[personi][x], tscale[personi][x]);
            print("p(t|x): "+str(ptx[personi][x]));
            px[personi][x] = numberOfTimesRouteTaken[personi][x]/sum(numberOfTimesRouteTaken[personi]);
            print("p(x): "+str(px[personi][x]));

        sumOfProbabilities = 0;
        for x in range(0,len(numberOfTimesRouteTaken)):    
            sumOfProbabilities = sumOfProbabilities + ptx[personi][x]*px[personi][x]; #this is p(t)

        for x in range(0,len(numberOfTimesRouteTaken)):     
            pbxt[personi][x] = (ptx[personi][x]*px[personi][x])/sumOfProbabilities;
            print("Route " + str(x) +" pb(x|t): "+str(pbxt[personi][x]));

        estimateInEachRegion = []
        #Estimated exposure
        for j in range(0,len(actualTimeInEachRegion)):
            estimateForThisRegion = 0;
            for i in range(0,len(numberOfTimesRouteTaken)): #len(numberOfTimesRouteTaken)= number of candidate paths
                estimateForThisRegion = estimateForThisRegion+percentOfTimeInEachRegion[i][j]*pathUserTime*pbxt[personi][i];
            estimateInEachRegion.append(estimateForThisRegion);


        print("Estimated time in each region: " + str(estimateInEachRegion));
        #c_i*t_i for our estimated t_i
        for regionNum in range(0,len(estimateInEachRegion)):
            estimatedCITI[personi][regionNum].append(ci[personi][regionNum]*estimateInEachRegion[regionNum]);
            print("Estimated CITI region "+ str(regionNum)+": "+ str(ci[personi][regionNum]*estimateInEachRegion[regionNum]))
            
                #update if we've reached the end
        if k == numberOfTest-1:
            for i in range(0,len(numberOfTimesRouteTaken[personi])): 
                #Update prior hyperparameters
                oldm = m[personi][i];
                oldv = v[personi][i];
                olda = a[personi][i];
                oldb = b[personi][i];

                v[personi][i] = oldv/(1+c[personi][i]*oldv);
                m[personi][i] = (oldm/oldv+c[personi][i]*tbar[personi][i])*v[personi][i]        
                a[personi][i] = olda+c[personi][i]/2;

                b[personi][i] = oldb + (1/2)*(pow(oldm,2)/oldv+sumtijsquared[personi][i]-pow(m[personi][i],2)/v[personi][i]);

                c[personi][i] = 0; 
                tbar[personi][i] = 0;
                sumtijsquared[personi][i] = 0;
                routeTravelTimes[personi][i] = [];

                print("Route "+str(i)+" UPDATED HYPER PARAMS \nm: "+str(m[personi][i])+
                "\nv: " + str(v[personi][i]) + 
                "\na: " + str(a[personi][i]) +
                "\nb: " + str(b[personi][i]));

                #Update Tdistribution params

                tloc[personi][i] = m[personi][i];
                tscale[personi][i] = math.sqrt(b[personi][i]*(1+v[personi][i])/(a[personi][i]));
                tdf[personi][i] = 2*a[personi][i];

#NO LONGER WORKS AFTER 100 PERSON UPDATE. LEAVING IN AS COMMENT FOR FUTURE REFERENCE.
#plot the average estimate compared to average actual    
#ind = numpy.arange(len(estimateInEachRegion)) #number of regions
#width = 0.35
#fig, ax = plt.subplots(1)
#actualStandardDeviations = []
#actualAverages = [];
#Find averages for each region
#for i in range(0, len(actualCITI)):
#    actualAverages.append(sum(actualCITI[i])/len(actualCITI[i]));
#    arr = numpy.array(actualCITI[i]);
#    std = numpy.std(arr, axis=0);
#    actualStandardDeviations.append(std);
#actualavg = ax.bar(ind, actualAverages, width, color='g', yerr=actualStandardDeviations)
#estimatedStandardDeviations = []
#estimatedAverages = [];
#Find averages for each region
#for i in range(0, len(actualCITI)):
#    estimatedAverages.append(sum(estimatedCITI[i])/len(estimatedCITI[i]));
#    arr = numpy.array(estimatedCITI[i]);
#    std = numpy.std(arr, axis=0);
#    estimatedStandardDeviations.append(std);
#estimatedavg = ax.bar(ind+width, actualAverages, width, color='b', yerr=estimatedStandardDeviations)

# add some text for labels, title and axes ticks
#ax.set_ylabel('Average CITI')
#ax.set_xlabel('Region')
#ax.set_title('Actual vs Estimated CITI for each Region')


#plt.show()



Path: 2
User Time in Seconds: 1594.7980396515156
Time in each region: [191.37576475818187, 191.37576475818187, 255.1676863442425, 95.68788237909094, 63.791921586060624, 191.37576475818187, 0.0, 0.0, 271.11566674075766, 0.0, 207.32374515469704, 111.6358627756061]
Actual CITI region 0: 1531.006118065455
Actual CITI region 1: 1913.7576475818187
Actual CITI region 2: 3317.1799224751526
Actual CITI region 3: 861.1909414118185
Actual CITI region 4: 1020.67074537697
Actual CITI region 5: 2296.5091770981826
Actual CITI region 6: 0.0
Actual CITI region 7: 0.0
Actual CITI region 8: 2711.1566674075766
Actual CITI region 9: 0.0
Actual CITI region 10: 2695.2086870110616
Actual CITI region 11: 1227.994490531667

Probabilities for t =1594.7980396515156
ROUTE 0 df = 4.0
p(t|x): 0.000122689768851
p(x): 0.10344827586206896
ROUTE 1 df = 9.0
p(t|x): 0.00372366927636
p(x): 0.27586206896551724
ROUTE 2 df = 5.0
p(t|x): 0.00167257165086
p(x): 0.1724137931034483
ROUTE 3 df = 9.0
p(t|x): 0.000906620959578
p(x)

In [36]:
#Stored Data
#allRouteTravelTimes[personNumber][routeNumber][i]  
#actualCITI[personi][regionNum][i]
#estimate

In [ ]:
#NO LONGER WORKS AFTER 100 PERSON UPDATE
#for i in range(0,len(numberOfTimesRouteTaken)):
#    fig, ax = plt.subplots(1, 1)
#    x = numpy.linspace(t.ppf(0.1, tdf[i], tloc[i], tscale[i]), t.ppf(0.9, tdf[i], tloc[i], tscale[i]), 100)
#    ax.plot(x, t.pdf(x, tdf[i],tloc[i], tscale[i]),'r-', lw=5, alpha=0.6, label='t pdf')    
#    ax.set_xlabel('Time (Seconds)')
#    ax.set_title('P(t|x) for Route ' + str(i))
#plt.show()